In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas_profiling

# Data Collection: Scraping from Baseball Reference (BR)
Baseball Reference is the only resource I found with historical data for the KBO preceding 2014. For thoroughness, I pulled all of the available data. There are two steps to the scraping because indivdual player season-to-season performance data is nested away on different individual pages of the website.

In [2]:
# scrape BR's KBO encyclopedia page for individual team/season page IDs
url = "https://www.baseball-reference.com/register/league.cgi?code=KBO&class=Fgn"
r = requests.get(url)
html_doc = r.text
regex = "\/register\/team\.cgi\?id=\w{8}"
matches = re.findall(regex, html_doc)
team_season_ids = [match[-8:] for match in matches]

In [3]:
# scrape each team/season page ID's table, and concatenate into one DataFrame
raw = pd.DataFrame()
base_url = "https://www.baseball-reference.com/register/team.cgi?id="
with tqdm(total=len(team_season_ids)) as pbar:    
    for id in team_season_ids:
        team_season_url = base_url + id
        html_doc = requests.get(team_season_url).text
        raw_new = pd.DataFrame(pd.read_html(html_doc)[0])
        raw_new['Team'] = str(BeautifulSoup(html_doc).title.string.split()[1]) + " " + str(BeautifulSoup(html_doc).title.string.split()[2])
        raw_new['Season'] = BeautifulSoup(html_doc).title.string.split()[0]
        raw = pd.concat([raw, raw_new], ignore_index=True)
        pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [14:06<00:00,  2.71s/it]


In [4]:
# save to local directory
raw.to_csv('KBO_Projections_Raw_Data.csv')

# Data Cleaning
The raw data has a variety of issues, which I address individually below.

In [5]:
df = pd.read_csv('KBO_Projections_Raw_Data.csv')
# delete rows that are team totals and not individual players
df.drop(df.loc[df['Rk'].isna()].index, inplace=True)
# delete cols that aren't substantive
df.drop(columns = ['Rk', 'Notes', 'Unnamed: 0'], inplace=True)

In [6]:
# extract batting stance (implied via special characters in name column)
Batting_Stances = []
for name in df['Name']:
    if '*' in name:
        Batting_Stances.append("Left-Handed")
    elif '#' in name:
        Batting_Stances.append("Switch Hitter")
    elif '?' in name:
        Batting_Stances.append("Unknown")
    else:
        Batting_Stances.append("Right-Handed")
df['Batting Stance'] = Batting_Stances

In [7]:
# remove what are now excess characters from name column
df['Name'] = df['Name'].str.replace("*","")
df['Name'] = df['Name'].str.replace("#","")
df['Name'] = df['Name'].str.replace("?","")

In [8]:
# fix one missing entry
df.loc[1651, 'Age'] = 24

In [9]:
# check for duplicates
print(df.duplicated().value_counts())

False    8070
dtype: int64


In [10]:
# check for "common sense" outliers (ignoring IBB, SB, CS)
print(df.describe())

                 G           PA           AB            R            H  \
count  8070.000000  8070.000000  8070.000000  8070.000000  8070.000000   
mean     60.409418   182.451549   159.260719    22.024287    42.583891   
std      43.385525   179.285612   155.726935    24.916590    46.103986   
min       1.000000     0.000000     0.000000     0.000000     0.000000   
25%      18.000000    22.000000    20.000000     2.000000     3.000000   
50%      57.000000   114.000000   101.000000    12.000000    24.000000   
75%     101.000000   327.000000   284.000000    36.000000    74.000000   
max     144.000000   672.000000   600.000000   135.000000   201.000000   

                2B           3B           HR          RBI           SB  ...  \
count  8070.000000  8070.000000  8070.000000  8070.000000  4534.000000  ...   
mean      7.351425     0.802107     3.950929    20.646964     3.861270  ...   
std       8.581665     1.468847     6.714054    25.374574     7.388046  ...   
min       0.00000

In [11]:
# because they make up <1% of the dataset, drop rows with na values in OBP, SLG, OPS
df = df.dropna(subset=['OBP', 'SLG', 'OPS'])

In [12]:
# create subsets of df containing only the numeric (and categorical) cols for ImperativeImputer
df = df.set_index('Name')
df_num_only = df.drop(columns=['Team', 'Batting Stance'])
df_categorical = df[['Team', 'Batting Stance']]

In [13]:
# data for SB, CS, and IBB wasn't recorded before 2001, so I opted to fill in missing values with IterativeImputer
imp = IterativeImputer(min_value=0)
imp.fit(df_num_only)
imputed_df = imp.transform(df_num_only)
imputed_df = pd.DataFrame(imputed_df, columns = df_num_only.columns, index=df_num_only.index)

In [14]:
# combine imputed_df into df with removed categorical variables
full_df = pd.concat([df_categorical, imputed_df], axis=1)

In [15]:
# change dtypes of columns
columns_to_int = ['Age','G','PA','AB','R','H','2B','3B','HR','RBI','SB','CS','BB','SO','TB','GDP','HBP','SH','SF','IBB','Season']
for col in columns_to_int:
    full_df[col] = full_df[col].astype(int)

# Data Export
With the data fully cleaned, export to a new CSV for future steps.

In [16]:
full_df.to_csv('KBO_Projections_Data_Clean.csv')